# 연관성 분석 측정 지표
상품 추천, 즉 고객이 함께 많이 구매할 것 같은 상품을 예측하는 데에 가장 많이 활용되는 척도로는 지지도( support), 신뢰도(Confidence), 향상도(Lift)라는 것이 있다.
- 지지도: 사과와 우유를 얼마나 함께 구매하는지를 나타내는 지표라고 볼 수 있습니다. 지지도는 좋은 규칙(구성비가 높고 빈도가 많은)을 찾거나 불필요한 연산을 줄이기 위한 기준으로 사용된다. A와 B를 모두 포함하는 거래의 수/ 전체 거래의 수
- 신뢰도: 사과를 많이 구매한 사람 중 얼마나 많은 사람이 우유도 함께 구매했을까요? 확률을 구해보니 사과를 구매한 사람은 3명이고, 그 중 2명이 우유를 구매했네요. 그럼 여기서 구할 수 있는 신뢰도는 66.7%이다. 지지도와 달리  confidence(A->B)의 값과 confidence(B->A)의 값이 다르고 값이 1에 가까울수록 연관성이 높다.
- 향상도 : 지지도가 별로 팔리지 않는 상품을 솎아내기 위한 지표였다면 거꾸로 많이 ㄹ팔리는 인기상품을 골라낼 때 활용되는 것이 향상도입니다. A를 구매하지 않았을 때 품목 B를 구매할 확률 대비 A를 구매했을 때 품목 B의 구매 확률의 증가 비율을 의미한다. (A와 B를 모두 포함하는 거래의 수 * 전체 거래의수) / (A를 포함하는 거래의 수 * B를 포함하는 거래의 수) 향상도가 1이면 두 품목은 독립이고 1보다 작다면 음의 상관관계로 A를 구매하면 B를 구매하지 않을 확률이 구매할 확률보다 큼 을 의미한다. 향상도가 1보다 크면 두 품목은 양의 상관관계로 임의로  B를 구매할 확률보다  A를 구매한 후 B를 구매할 확률이 큼을 의미한다.

향상도 ( lift )
- 품목 B를 구매한 고객 대비 품목 A를 구매한 후 품목 B를 구매하는 고객에 대한 확률
- 연관규칙 A -> B는 품목 A와 품목 B가 서로 관련이 없는 경우 (독립적인 경우) 에 P(B|A) = P(B)이므로 향상도는 1. (연관성이 없는 서로 독립적인 관계를 의미한다.)
- 향상도가 1보다 크면 이 규칙은 결과를 예측하는데 있어서 우수, 연관성이 높다는걸 반영
- 향상도가 1보다 작으면 우연적 기회보다 도움이 되지 않는 다는 의미. 연관성이 없다는 것을 반영

In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [4]:
dataset=[['사과','치즈','생수'],
        ['생수','호두','치즈','고등어'],
        ['수박','사과','생수'],
        ['생수','호두','치즈','옥수수']]
te= TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary,columns=te.columns_)
df

,고등어,사과,생수,수박,옥수수,치즈,호두
0,False,True,True,False,False,True,False
1,True,False,True,False,False,True,True
2,False,True,True,True,False,False,False
3,False,False,True,False,True,True,True


In [11]:
# 지지도 0.5
frequent_itemsets = apriori(df,min_support=0.5,use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.50,(사과)
1,1.00,(생수)
2,0.75,(치즈)
3,0.50,(호두)
4,0.50,"(생수, 사과)"
5,0.75,"(치즈, 생수)"
6,0.50,"(생수, 호두)"
7,0.50,"(치즈, 호두)"
8,0.50,"(치즈, 생수, 호두)"


In [12]:
# 신뢰도 0.3
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric='confidence',min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(생수),(사과),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
1,(사과),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
2,(치즈),(생수),0.75,1.00,0.75,1.000000,1.000000,0.000,inf
3,(생수),(치즈),1.00,0.75,0.75,0.750000,1.000000,0.000,1.0
4,(생수),(호두),1.00,0.50,0.50,0.500000,1.000000,0.000,1.0
5,(호두),(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf
6,(치즈),(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
7,(호두),(치즈),0.50,0.75,0.50,1.000000,1.333333,0.125,inf
8,"(치즈, 생수)",(호두),0.75,0.50,0.50,0.666667,1.333333,0.125,1.5
9,"(치즈, 호두)",(생수),0.50,1.00,0.50,1.000000,1.000000,0.000,inf


# 과제

In [13]:
dataset = [
    ['milk','Onion','Nutmeg','Eggs','Yogurt'],
    ['Onion','Nutmeg','Eggs','Yogurt'],
    ['Milk','Apple','Eggs'],
    ['Milk','Unicorn','Corn','Yogurt'],
    ['Corn','Onion','Onion','Ice Cream','Eggs']
]

In [14]:
te = TransactionEncoder()
te_ary = te.fit_transform(dataset)
df = pd.DataFrame(te_ary,columns=te.columns_)
df

,Apple,Corn,Eggs,Ice Cream,Milk,Nutmeg,Onion,Unicorn,Yogurt,milk
0,False,False,True,False,False,True,True,False,True,True
1,False,False,True,False,False,True,True,False,True,False
2,True,False,True,False,True,False,False,False,False,False
3,False,True,False,False,True,False,False,True,True,False
4,False,True,True,True,False,False,True,False,False,False


In [17]:
# 지지도 0.5
frequent_itemsets = apriori(df,min_support=0.5,use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.8,(Eggs)
1,0.6,(Onion)
2,0.6,(Yogurt)
3,0.6,"(Eggs, Onion)"


In [19]:
# 신뢰도 0.3
association_rules(frequent_itemsets,metric='confidence',min_threshold=0.3)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf


### 연관성 분석
- 연관성 분석은 상품 혹은 서비스 간의 관계를 살펴보고 이로부터 유용한 규칙을 찾아내고자 할 때 이용될 수 있는 기법
- 동시 구매 될 가능성이 큰 상품들을 찾아내는 기법으로 장바구니 분석과 관련된 문제에 많이 적용된다
- 측정의 기본은 얼마나 자주 구매되었는가 하는 빈도를 기본으로 연관정도를 |정량화하기 위해서는 지지도, 신뢰도, 향상도를 계산하여 기준으로 함
- 연관성 규칙의 기본적인 개념은 장바구니 품목들을 식별하는 것에서부터 시작됨.
- 사건들은 동시자발적으로 발생하며 이러한 사건들은 상호 영향을 주면서 결과가 나타나게 되는데 이와 같이 사건 또는 품목 간에 일어나는 연관성을 규명하려는 것이 연관성 규칙이다.

    - 지지도 (Support) : 전체에서 A | B 를 구매할 확률 -- 지지도가 높아야 어느정도 판매되는 상품이라고 판단할 수 있음 (필터링- 잘 안팔리는 것들을 거름)
    - 신뢰도 (Confidence) : A를 산 사람이 B 상품을 구매할 확률 ( B를 산 사람이 A 상품을 구매할 확률 간의 차기아 있음.) 신뢰도가 높다는 것은 연관성이 높다는 지표.
    - 향상도 (Lift) :  관계가 있는지 , 독립적인지 판단. 향상도가 1이면 아무런 연관이 없음 (독립) / 1보다 크면 영향이 높다고 판단함.(신뢰도와 향상도의 관계 생각) 1보다 작으면 연관성이 없다고 봄 (Random보다 낮은 관계를 의미. (음의 상관관계))